In [175]:
import numpy as np
import pandas

def d20():
    return np.random.randint(1,21)

def do_rounds_and_update(a,b,rounds,df):
    wins = do_rounds(a,b,rounds)
    a_wins = wins[0]
    b_wins = wins[1]
    a_id = a[0]
    b_id = b[0]
    df.at[a_id,'total_wins'] = df.at[a_id,'total_wins'] + a_wins
    df.at[b_id,'total_wins'] = df.at[b_id,'total_wins'] + b_wins
    if a_wins > b_wins:
        df.at[a_id,'pair_wins'] = df.at[a_id,'pair_wins'] + 1
    else:
        df.at[b_id,'pair_wins'] = df.at[b_id,'pair_wins'] + 1
    return creatures

def do_rounds(a,b,rounds):
    a_wins = 0
    b_wins = 0
    for _ in np.arange(rounds):
        wins = do_round(a,b)
        a_wins = a_wins + wins[0]
        b_wins = b_wins + wins[1]
    print ("After %d rounds %s won %d times and %s won %d times" % (rounds,a.name,a_wins,b.name,b_wins))
    return [a_wins,b_wins]

def do_round( a, b ):
    a_init = d20() + a.init
    if debug:
        print("%s rolled %d init" % (a.name,a_init))
    b_init = d20() + b.init
    if debug:
        print("%s rolled %d init" % (b.name,b_init))
    if (a_init >= b_init):
        attacker = a
        target = b
    else:
        attacker = b
        target = a
    round = 0
    while (attacker.hp > 0 and target.hp > 0):
        if debug:
            print("    Round %d: %s hp is %d and %s hp is %d" % (round, attacker.name,attacker.hp,target.name,target.hp))
        round = round + 1
        attacker,target = do_attack(attacker,target)
        if (target.hp <= 0):
            if debug:
                print("%s is dead" % target.name)
        else:
            target,attacker = do_attack(target,attacker)
            if (attacker.hp <= 0):
                if debug:
                    print("%s is dead" % attacker.name)
    if (target.hp <= 0):
        return [1,0]
    else:
        return [0,1]
    
def do_attack( attacker, target ):
    if debug:
        print ("        %s makes %d attacks against %s" % (attacker.name, attacker.att_num, target.name))
    for _ in np.arange(attacker.att_num):
        attack = d20() + attacker.hit_mod
        if (attack >= target.ac):
            target.hp = target.hp - attacker.dam_avg
            if debug:
                print("            Attacker %s hits %s for %d damage" % (attacker.name, target.name, attacker.dam_avg ))
        else:
            if debug:
                print("            Attacker %s misses %s" % (attacker.name, target.name))
    return [attacker, target]

In [202]:
debug = False
rounds = 300
creatures = pandas.read_csv('creatures.csv')
rows = creatures.shape[0]
creatures['pair_wins'] = np.zeros(rows)
creatures['total_wins'] = np.zeros(rows)
a = creatures.sample(1).to_records()[0]
b = creatures.sample(1).to_records()[0]
creatures = do_rounds_and_update(a,b,rounds,creatures)
creatures.loc[creatures['total_wins'] > 0]

After 300 rounds Hippogriff won 184 times and Djinni won 116 times


,name,cr,ac,hp,init,att_num,hit_mod,dam_avg,pair_wins,total_wins
84,Djinni,11.0,17,161,2.15,3,5,10.0,0.0,116.0
167,Hippogriff,1.0,11,19,1.13,2,5,7.0,1.0,184.0
